# Transfer Learning 
#### Here, we train model to detect pneumonia using transfer learning with the help of MobileNet, Inception, ResNet50, and ResNet152 

#### In this notebook, we do the following :-
1. Define basic parameters for image processing like image_size and batch_size
2. Create a tensorflow data pipeline to efficiently read the images while training the model
3. Train preloaded MobileNet model - a) by freezing the CNN parameters, b) by fine-tuning the CNN parameters
3. Train preloaded ResNet50 model - a) by freezing the CNN parameters, b) by fine-tuning the CNN parameters
4. Train ResNet152 - a) by freezing the CNN parameters (limited computational resource to train all parameters)
5. Train InceptionV3 - a) by freezing the CNN parameters, b) by fine-tuning the CNN parameters

<i>The performance analysis of all these trained models is done in "d) Model Performance Analysis" notebook</i><br>

#### Transfer Learning is done such that,
1. When the layers are frozen, we apply a fresh trainable Conv2D layer followed by fully connected layers to get the output
2. When the layers are trainable, we directly connect them to fresh fully connected layers to get the output

#### The data pipeline does the following :-
1. Reads a "batch" of image file_paths from the metadata
2. Read those images and processes it - resizing, casting into appropiate datatype, image_augmentation (only in training)
4. Return the batch of processed images for feeding into the model for training or validation
5. The data pipeline shuffles the training set while creating a batch to train the model to avoid ordering bias
6. It also automatically determines the appropiate #threads while mapping the file path to output image to save time
7. It prefetches the next batch while the current batch is feed into the model for training to save time

#### Following callback methods were used while training :-
1. <b>ReduceLROnPlateau</b> - this will reduce the learning rate by some factor if the monitored metric (val_loss) decreases twice in epochs while training. This will avoid overfitting
2. <b>ModelCheckpoint</b> - this saves the model after each epoch. This will allow us to use any intermediate epoch model if needed
3. <b>CSVLogger</b> - this saves the model performance metrics after each epoch in a csv file for later analysis
4. <b>EarlyStopping</b> - this will stop training of the model if monitored metric (val_loss) decreases 3 times in a row. This will avoid overfitting




In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, GlobalMaxPooling2D, ReLU, GlobalAveragePooling2D, Rescaling
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, CSVLogger, EarlyStopping


# Parameters for Image Processing

In [4]:
batch_size = 12
IMG_SIZE = 224

# Data Pipeline

### Prepare metadata for images

In [5]:
train_df = pd.read_csv('Data/pneumonia/train_metadata.csv')
train_df['file_path'] = 'Data/pneumonia/Training/Images/' + train_df['patientId'] + '.png'
train_data = list(zip(train_df['file_path'], train_df['Target']))
train_paths, train_target = zip(*train_data)


validation_df = pd.read_csv('Data/pneumonia/val_metadata.csv')
validation_df['file_path'] = 'Data/pneumonia/Training/Images/' + validation_df['patientId'] + '.png'
validation_data = list(zip(validation_df['file_path'], validation_df['Target']))
validation_paths, validation_target = zip(*validation_data)

print(train_df.shape)
print(validation_df.shape)

(25727, 12)
(2250, 12)


### Mapping Function for Image Processing

In [ ]:
# Data Augmentation block
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomContrast

augmentation_block = tf.keras.Sequential([
    RandomFlip("vertical"),
], name="data_augmentation")


In [7]:

def load_and_process_train(image_path, target):

    #read file
    image = tf.io.read_file(image_path)

    #process image
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE], method = 'bilinear')
    image = tf.cast(image, tf.float32)
    image = tf.image.grayscale_to_rgb(image) #to convert from 1 channels to 3 channels
    
    #augmentation
    image = augmentation_block(image, training=True)
    
    return image, target

def load_and_process_validation(image_path, target):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE], method = 'bilinear')
    image = tf.cast(image, tf.float32)
    image = tf.image.grayscale_to_rgb(image) #to convert from 1 channels to 3 channels

    return image, target


### Create Tensorflow Pipeline

In [8]:
# TF datasets
train_ds = tf.data.Dataset.from_tensor_slices((list(train_paths), list(train_target)))
train_ds = train_ds.map(load_and_process_train, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(100).batch(batch_size).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((list(validation_paths),  list(validation_target)))
val_ds = val_ds.map(load_and_process_validation, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)


# Compile and Run

In [9]:

def model_compile_and_fit(model,model_name):

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), \
                  loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall(), AUC(name='auc')])

    os.makedirs(f"final_models/{model_name}", exist_ok=True)
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1, patience=2, min_lr=1e-6,verbose=1)
    
    checkpoint_cb = ModelCheckpoint(filepath=f"final_models/{model_name}/{model_name}_{{epoch:02d}}.keras",save_freq='epoch', \
                                    save_weights_only=True,save_best_only=False,verbose=1)
    
    csv_logger = CSVLogger(f"final_models//{model_name}/{model_name}_log.csv", append=True)
    
    early_stop = EarlyStopping(monitor='val_loss',patience=3,mode='min', verbose = 1)
    
    model.fit(train_ds, validation_data=val_ds, epochs=20, verbose = 1, \
            callbacks=[checkpoint_cb, reduce_lr,csv_logger, early_stop])
    

# Modelling

### Convolution and Fully Connected Layers

In [10]:
def fc_layer(x):
    
    x = GlobalMaxPooling2D()(x) #512
    x = Dropout(0.3)(x)
    
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    x = Dense(30, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    x = Dense(1, activation='sigmoid')(x)

    return x


# MobileNet

In [12]:
mobilenet = tf.keras.applications.MobileNet(
    input_shape = (224, 224, 3),
    include_top = False,
    weights = 'imagenet')


### Train all layers

In [18]:
mobilenet.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
rescaled_inputs = Rescaling(scale = 2, offset = -1)(inputs) #taking pixels from [0,1] to [-1,1]
mobilenet_x = mobilenet(rescaled_inputs)
outputs = fc_layer(mobilenet_x)
mobilenet_model = Model(inputs = inputs, outputs = outputs)


In [19]:
model_compile_and_fit(mobilenet_model, 'mobilenet_trainable')

Epoch 1/20
2144/2144 [==============================] - ETA: 0s - loss: 0.6082 - accuracy: 0.7347 - precision: 0.5873 - recall: 0.5554 - auc: 0.7689
Epoch 1: saving model to final_models/mobilenet_trainable\mobilenet_trainable_01.keras
2144/2144 [==============================] - 207s 92ms/step - loss: 0.6082 - accuracy: 0.7347 - precision: 0.5873 - recall: 0.5554 - auc: 0.7689 - val_loss: 0.5282 - val_accuracy: 0.7631 - val_precision: 0.7973 - val_recall: 0.3329 - val_auc: 0.8388 - lr: 1.0000e-04
Epoch 2/20
2144/2144 [==============================] - ETA: 0s - loss: 0.4500 - accuracy: 0.7911 - precision: 0.6949 - recall: 0.6106 - auc: 0.8499
Epoch 2: saving model to final_models/mobilenet_trainable\mobilenet_trainable_02.keras
2144/2144 [==============================] - 190s 89ms/step - loss: 0.4500 - accuracy: 0.7911 - precision: 0.6949 - recall: 0.6106 - auc: 0.8499 - val_loss: 0.4524 - val_accuracy: 0.7844 - val_precision: 0.8294 - val_recall: 0.3977 - val_auc: 0.8623 - lr: 1.000

### Freeze layers

In [13]:
mobilenet.trainable = False
inputs = tf.keras.Input(shape=(224, 224, 3))
rescaled_inputs = Rescaling(scale = 2, offset = -1)(inputs)
x = mobilenet(rescaled_inputs)
x = Conv2D(512, (3,3), padding = 'same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
outputs = fc_layer(x)
mobilenet_model_untrainable = Model(inputs = inputs, outputs = outputs)


In [14]:
model_compile_and_fit(mobilenet_model_untrainable, 'mobilenet_untrainable')

Epoch 1/20
2144/2144 [==============================] - ETA: 0s - loss: 0.5693 - accuracy: 0.7264 - precision: 0.5921 - recall: 0.4470 - auc: 0.7393
Epoch 1: saving model to final_models/mobilenet_untrainable\mobilenet_untrainable_01.keras
2144/2144 [==============================] - 115s 49ms/step - loss: 0.5693 - accuracy: 0.7264 - precision: 0.5921 - recall: 0.4470 - auc: 0.7393 - val_loss: 0.5874 - val_accuracy: 0.7320 - val_precision: 0.6268 - val_recall: 0.3695 - val_auc: 0.7146 - lr: 1.0000e-04
Epoch 2/20
2144/2144 [==============================] - ETA: 0s - loss: 0.5087 - accuracy: 0.7588 - precision: 0.6508 - recall: 0.5196 - auc: 0.7912
Epoch 2: saving model to final_models/mobilenet_untrainable\mobilenet_untrainable_02.keras
2144/2144 [==============================] - 102s 47ms/step - loss: 0.5087 - accuracy: 0.7588 - precision: 0.6508 - recall: 0.5196 - auc: 0.7912 - val_loss: 0.5599 - val_accuracy: 0.7347 - val_precision: 0.5989 - val_recall: 0.4781 - val_auc: 0.7589 - l

# ResNet50

In [11]:
from tensorflow.keras.applications import resnet

resnet50 = tf.keras.applications.ResNet50(
    input_shape = (224, 224, 3),
    include_top = False, #weather to include the FC layer at the end
    weights = 'imagenet') #whather to initialize the weights particular to some dataset


### Train all layers

In [13]:

resnet50.trainable = True

inputs = Input(shape=(224, 224, 3))
scaled_inputs_for_preprocessing = Rescaling(scale = 255)(inputs)
preprocessed_inputs = resnet.preprocess_input(scaled_inputs_for_preprocessing)
resnet50_x = resnet50(preprocessed_inputs)
outputs = fc_layer(resnet50_x)
resnet50_model = Model(inputs = inputs, outputs = outputs)


In [14]:
model_compile_and_fit(resnet50_model, 'resnet50_trainable')

Epoch 1/20
2144/2144 [==============================] - ETA: 0s - loss: 0.6377 - accuracy: 0.7363 - precision: 0.5933 - recall: 0.5413 - auc: 0.7763
Epoch 1: saving model to final_models/resnet50_trainable\resnet50_trainable_01.keras
2144/2144 [==============================] - 589s 268ms/step - loss: 0.6377 - accuracy: 0.7363 - precision: 0.5933 - recall: 0.5413 - auc: 0.7763 - val_loss: 0.4893 - val_accuracy: 0.7560 - val_precision: 0.7516 - val_recall: 0.3371 - val_auc: 0.8356 - lr: 1.0000e-04
Epoch 2/20
2144/2144 [==============================] - ETA: 0s - loss: 0.4827 - accuracy: 0.7793 - precision: 0.7157 - recall: 0.5068 - auc: 0.8367
Epoch 2: saving model to final_models/resnet50_trainable\resnet50_trainable_02.keras
2144/2144 [==============================] - 571s 266ms/step - loss: 0.4827 - accuracy: 0.7793 - precision: 0.7157 - recall: 0.5068 - auc: 0.8367 - val_loss: 0.6309 - val_accuracy: 0.7138 - val_precision: 0.6012 - val_recall: 0.2722 - val_auc: 0.6399 - lr: 1.0000e

### Freeze layers

In [12]:
resnet50.trainable = False

inputs = Input(shape=(224, 224, 3))
scaled_inputs_for_preprocessing = Rescaling(scale = 255)(inputs)
preprocessed_inputs = resnet.preprocess_input(scaled_inputs_for_preprocessing)
resnet50_x = resnet50(preprocessed_inputs)
x = Conv2D(512, (3,3), padding = 'same')(resnet50_x)
x = BatchNormalization()(x)
x = ReLU()(x)
outputs = fc_layer(x)
resnet50_model_untrainable = Model(inputs = inputs, outputs = outputs)


In [13]:
model_compile_and_fit(resnet50_model_untrainable, 'resnet50_untrainable')

Epoch 1/20
2144/2144 [==============================] - ETA: 0s - loss: 0.5651 - accuracy: 0.7100 - precision: 0.5643 - recall: 0.3825 - auc: 0.7246
Epoch 1: saving model to final_models/resnet50_untrainable\resnet50_untrainable_01.keras
2144/2144 [==============================] - 275s 124ms/step - loss: 0.5651 - accuracy: 0.7100 - precision: 0.5643 - recall: 0.3825 - auc: 0.7246 - val_loss: 0.6114 - val_accuracy: 0.7476 - val_precision: 0.5949 - val_recall: 0.6234 - val_auc: 0.7939 - lr: 1.0000e-04
Epoch 2/20
2144/2144 [==============================] - ETA: 0s - loss: 0.5180 - accuracy: 0.7449 - precision: 0.6316 - recall: 0.4733 - auc: 0.7787
Epoch 2: saving model to final_models/resnet50_untrainable\resnet50_untrainable_02.keras
2144/2144 [==============================] - 259s 121ms/step - loss: 0.5180 - accuracy: 0.7449 - precision: 0.6316 - recall: 0.4733 - auc: 0.7787 - val_loss: 0.5687 - val_accuracy: 0.7360 - val_precision: 0.5592 - val_recall: 0.7659 - val_auc: 0.8119 - lr:

# ResNet152

In [11]:
from tensorflow.keras.applications import resnet

resnet152 = tf.keras.applications.ResNet152(
    input_shape = (224, 224, 3),
    include_top = False, 
    weights = 'imagenet')


### Train all layers

In [12]:
resnet152.trainable = True

inputs = Input(shape=(224, 224, 3))
scaled_inputs_for_preprocessing = Rescaling(scale = 255)(inputs)
preprocessed_inputs = resnet.preprocess_input(scaled_inputs_for_preprocessing)
resnet152_x = resnet152(preprocessed_inputs)
outputs = fc_layer(resnet152_x)
resnet152_model = Model(inputs = inputs, outputs = outputs)


In [ ]:
model_compile_and_fit(resnet152_model, 'resnet152_trainable')

### Freeze Layers

In [12]:
resnet152.trainable = False

inputs = Input(shape=(224, 224, 3))
scaled_inputs_for_preprocessing = Rescaling(scale = 255)(inputs)
preprocessed_inputs = resnet.preprocess_input(scaled_inputs_for_preprocessing)
resnet152_x = resnet152(preprocessed_inputs)
x = Conv2D(512, (3,3), padding = 'same')(resnet152_x)
x = BatchNormalization()(x)
x = ReLU()(x)
outputs = fc_layer(x)
resnet152_untrainable_model = Model(inputs = inputs, outputs = outputs)


In [13]:
model_compile_and_fit(resnet152_untrainable_model, 'resnet152_untrainable_model')

Epoch 1/20
2144/2144 [==============================] - ETA: 0s - loss: 0.5645 - accuracy: 0.7113 - precision: 0.5624 - recall: 0.4122 - auc: 0.7291
Epoch 1: saving model to final_models/resnet152_untrainable_model\resnet152_untrainable_model_01.keras
2144/2144 [==============================] - 554s 253ms/step - loss: 0.5645 - accuracy: 0.7113 - precision: 0.5624 - recall: 0.4122 - auc: 0.7291 - val_loss: 0.4911 - val_accuracy: 0.7756 - val_precision: 0.6903 - val_recall: 0.5219 - val_auc: 0.8230 - lr: 1.0000e-04
Epoch 2/20
2144/2144 [==============================] - ETA: 0s - loss: 0.5184 - accuracy: 0.7447 - precision: 0.6239 - recall: 0.4947 - auc: 0.7809
Epoch 2: saving model to final_models/resnet152_untrainable_model\resnet152_untrainable_model_02.keras
2144/2144 [==============================] - 535s 250ms/step - loss: 0.5184 - accuracy: 0.7447 - precision: 0.6239 - recall: 0.4947 - auc: 0.7809 - val_loss: 0.4823 - val_accuracy: 0.7862 - val_precision: 0.6686 - val_recall: 0.

# InceptionV3

In [12]:

inceptionv3 = tf.keras.applications.InceptionV3(
    input_shape = (IMG_SIZE, IMG_SIZE, 3), #299
    include_top = False,
    weights = 'imagenet')


### Train all layers

In [12]:

inceptionv3.trainable = True

inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
rescaled_inputs = Rescaling(scale = 2, offset = -1)(inputs)
inceptionv3_x = inceptionv3(rescaled_inputs)
outputs = fc_layer(inceptionv3_x)
inceptionv3_model = Model(inputs = inputs, outputs = outputs)


In [13]:
model_compile_and_fit(inceptionv3_model, 'inceptionv3_trainable')

Epoch 1/20
2144/2144 [==============================] - ETA: 0s - loss: 0.4727 - accuracy: 0.7812 - precision: 0.6766 - recall: 0.5966 - auc: 0.8335
Epoch 1: saving model to final_models/inceptionv3_trainable\inceptionv3_trainable_01.keras
2144/2144 [==============================] - 779s 354ms/step - loss: 0.4727 - accuracy: 0.7812 - precision: 0.6766 - recall: 0.5966 - auc: 0.8335 - val_loss: 0.5319 - val_accuracy: 0.8067 - val_precision: 0.7276 - val_recall: 0.6178 - val_auc: 0.8710 - lr: 1.0000e-04
Epoch 2/20
2144/2144 [==============================] - ETA: 0s - loss: 0.4050 - accuracy: 0.8209 - precision: 0.7410 - recall: 0.6709 - auc: 0.8812
Epoch 2: saving model to final_models/inceptionv3_trainable\inceptionv3_trainable_02.keras
2144/2144 [==============================] - 759s 354ms/step - loss: 0.4050 - accuracy: 0.8209 - precision: 0.7410 - recall: 0.6709 - auc: 0.8812 - val_loss: 0.5236 - val_accuracy: 0.7538 - val_precision: 0.8355 - val_recall: 0.2722 - val_auc: 0.8619 -

### Freeze layers

In [13]:
inceptionv3.trainable = False

inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
rescaled_inputs = Rescaling(scale = 2, offset = -1)(inputs)
inceptionv3_x = inceptionv3(rescaled_inputs)
x = Conv2D(512, (3,3), padding = 'same')(inceptionv3_x)
x = BatchNormalization()(x)
x = ReLU()(x)
outputs = fc_layer(x)
inceptionv3_model_untrainable = Model(inputs = inputs, outputs = outputs)


In [14]:
model_compile_and_fit(inceptionv3_model_untrainable, 'inceptionv3_model_untrainable')

Epoch 1/20
2144/2144 [==============================] - ETA: 0s - loss: 0.5352 - accuracy: 0.7406 - precision: 0.6196 - recall: 0.4760 - auc: 0.7717
Epoch 1: saving model to final_models/inceptionv3_model_untrainable\inceptionv3_model_untrainable_01.keras
2144/2144 [==============================] - 323s 145ms/step - loss: 0.5352 - accuracy: 0.7406 - precision: 0.6196 - recall: 0.4760 - auc: 0.7717 - val_loss: 0.5838 - val_accuracy: 0.7440 - val_precision: 0.5874 - val_recall: 0.6305 - val_auc: 0.7924 - lr: 1.0000e-04
Epoch 2/20
2144/2144 [==============================] - ETA: 0s - loss: 0.4905 - accuracy: 0.7646 - precision: 0.6650 - recall: 0.5222 - auc: 0.8088
Epoch 2: saving model to final_models/inceptionv3_model_untrainable\inceptionv3_model_untrainable_02.keras
2144/2144 [==============================] - 306s 142ms/step - loss: 0.4905 - accuracy: 0.7646 - precision: 0.6650 - recall: 0.5222 - auc: 0.8088 - val_loss: 0.5834 - val_accuracy: 0.7493 - val_precision: 0.5989 - val_re